In [30]:
from ebooklib import utils
import os
from multiprocessing import Pool
import time
from bs4 import BeautifulSoup

In [27]:
d="/media/some/sophie-hhd/books/thrill/"
ls=!find $d -iname *.htm
lst=!ls $d*.htm*
len(ls),len(lst)
ffil=!find /media/some/sophie-hhd/books/htm2epub/ -iname *.html

In [31]:
#remove html tags from string html     
def html2txt(html):
    soup = BeautifulSoup(html,"lxml")
    return soup.get_text()

In [23]:
def mvht(l):
    ddir="/media/some/sophie-hhd/books/htm2epub/"
    m,f=os.path.split(l)
    ff=ddir+f[:-3]+"epub"
    fh=ddir+f
    fe=m+"/"+f[:-3]+"epub"
    if not os.path.exists(ff) and not os.path.exists(fe):
        tmp="/tmp/"+f[:-3]+"epub"
        tmpl="/tmp/"+f[:-3]+"htm"
        !cp "$l" "$tmpl"
        try:
            !ebook-convert "$tmpl" "$tmp" >/dev/null
            !rm "$tmpl"
            !mv "$tmp" "$ff"
        except:
            os.remove(l)
    
    !mv "$l" "$fh"
 

In [32]:
t0 = time.time()
p = Pool(5)
p.map(mvht, list(reversed(ls)))
p.terminate()
p.close()
t1 = time.time()
total = t1-t0
print('time: ',total)

mv: cannot stat ‘/media/some/sophie-hhd/books/thrill/HammerDown.htm’: No such file or directory
mv: cannot stat ‘/media/some/sophie-hhd/books/thrill/Phoenix Rising I - Morgana de Winter, Marie Harte, Michelle M. Pillow, Sherrill Quinn, Alicia Sparks.htm’: No such file or directory
mv: cannot stat ‘/media/some/sophie-hhd/books/thrill/Asimov's - Winter Fire.htm’: No such file or directory
mv: cannot stat ‘/media/some/sophie-hhd/books/thrill/Asimov's - Standing Room Only.htm’: No such file or directory
mv: cannot stat ‘/media/some/sophie-hhd/books/thrill/ClaimMe.htm’: No such file or directory
mv: cannot stat ‘/media/some/sophie-hhd/books/thrill/R.S.V.P. - Adair, Oh, Dunne.htm’: No such file or directory
mv: cannot stat ‘/media/some/sophie-hhd/books/thrill/Relic - Ellen Ashe,Celia Ashley,Annalee Blysse,Tracy L Ranson.htm’: No such file or directory
mv: cannot stat ‘/media/some/sophie-hhd/books/thrill/BoardResolution.htm’: No such file or directory
mv: cannot stat ‘/media/some/sophie-hhd/b

In [15]:
os.path.split(ls[147])

('/media/some/sophie-hhd/books/thrill/Romance Novels',
 'Georgette Heyer - The Grand Sophy (v2).htm')